In [6]:
import os
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Flatten
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

# Load data from directory of PNG images
data_dir = "D:\QA\Documents\College\Codes\project\\new\word_images"
labels = os.listdir(data_dir)
num_classes = len(labels)

X = []
y = []
for i, label in enumerate(labels):
    for filename in os.listdir(os.path.join(data_dir, label)):
        img_path = os.path.join(data_dir, label, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (28, 28))
        X.append(img)
        y.append(i)
        
X = np.array(X, dtype=np.float32)
X /= 255.0
y = np.array(y, dtype=np.int32)

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2)

word_dict = {i: label for i, label in enumerate(labels)}

count = np.zeros(num_classes, dtype='int')
for i in y:
    count[i] +=1

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.barh(labels, count)

plt.xlabel('Number of elements')
plt.ylabel('Words')
plt.grid()
plt.show()

shuff = shuffle(train_x[:100])

fig, ax = plt.subplots(3, 3, figsize=(10, 10))
axes = ax.flatten()

for i in range(9):
    axes[i].imshow(np.reshape(shuff[i], (28, 28)), cmap='Greys')
plt.show()

train_X = train_x.reshape(train_x.shape[0], train_x.shape[1], train_x.shape[2])
print("New shape of train data: ", train_X.shape)

test_X = test_x.reshape(test_x.shape[0], test_x.shape[1], test_x.shape[2])
print("New shape of test data: ", test_X.shape)


# Now we reshape the train & test image dataset so that they can be put in the model.

# New shape of train data:  (4000, 28, 28)
# New shape of train data:  (1000, 28, 28)

train_yOHE = to_categorical(train_y, num_classes=num_classes, dtype='int')
print("New shape of train labels: ", train_yOHE.shape)

test_yOHE = to_categorical(test_y, num_classes=num_classes, dtype='int')
print("New shape of test labels: ", test_yOHE.shape)

# define the RNN model architecture
model = Sequential()
model.add(LSTM(units=128, return_sequences=True, input_shape=(28, 28)))
model.add(Dropout(0.5))
model.add(LSTM(units=64))
model.add(Dropout(0.5))
model.add(Dense(units=num_classes, activation='softmax'))

# compile the model with appropriate loss and optimizer functions
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# print the model summary
model.summary()


model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_X, train_yOHE, epochs=10, validation_data=(test_X, test_yOHE))

model.summary()
model.save('model_hand.h5')

print("The validation accuracy is :", history.history['val_accuracy'])
print("The training accuracy is :", history.history['accuracy'])
print("The validation loss is :", history.history['val_loss'])
print("The training loss is :", history.history['loss'])

NotADirectoryError: [WinError 267] The directory name is invalid: 'D:\\QA\\Documents\\College\\Codes\\project\\new\\word_images\\hello.jpg'